In [35]:
import pandas as pd
import numpy as np
import cv2
import importlib
SEED = 1234
np.random.seed(SEED) 

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.utils import plot_model
import mylibs.VggNet as VggNet
import mylibs.ResNet as ResNet
import mylibs.SENet as SENet
importlib.reload(VggNet)
importlib.reload(ResNet)
importlib.reload(SENet)
importlib.reload(keras)
from keras.models import Model
from keras.layers import Input, Average
from keras import backend as K
from keras.layers.merge import Concatenate
from keras.engine import Layer
import tensorflow as tf
from sklearn.model_selection import train_test_split 
batch_size = 64

from sklearn.model_selection import StratifiedKFold

In [2]:
%cd E:\kaggle\iceberg

E:\kaggle\iceberg


In [3]:
def get_scaled_imgs(df):
    imgs = []
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

        imgs.append(np.dstack((a, b, c)))

    return np.array(imgs)

def get_more_images(imgs):
    more_images = []
    vert_flip_imgs = []
    hori_flip_imgs = []
    vh_flip_imgs = []
      
    for i in range(0,imgs.shape[0]):
        vert_flip_imgs.append(cv2.flip(imgs[i], 1))
        hori_flip_imgs.append(cv2.flip(imgs[i], 0))
        vh_flip_imgs.append(cv2.flip(imgs[i], -1))
      
    v = np.array(vert_flip_imgs)
    h = np.array(hori_flip_imgs)
    vh = np.array(vh_flip_imgs)
       
    more_images = np.concatenate((imgs,v,h, vh))
    
    return more_images

In [4]:
use_custom_augmentation = True
if use_custom_augmentation:
    df_train = pd.read_json('E:/kaggle/iceberg/train.json/data/processed/train.json')
    df_test = pd.read_json('E:/kaggle/iceberg/test.json/data/processed/test.json')
    Xtrain = get_scaled_imgs(df_train)
    Xtest = get_scaled_imgs(df_test)
    Ytrain = np.array(df_train['is_iceberg'])
    
    df_train["inc_angle"] = df_train["inc_angle"].replace('na',0)
    df_test["inc_angle"] = df_test["inc_angle"].replace('na',0)
    idx_tr = np.where(df_train["inc_angle"]>0)
    Xtrain = Xtrain[idx_tr[0]]
    Ytrain = Ytrain[idx_tr[0]]
    
    Xtrain = get_more_images(Xtrain) 
    Ytrain = np.concatenate((Ytrain,Ytrain,Ytrain, Ytrain))

In [38]:
class ImgCrop(Layer):
    def __init__(self, img_size=75, batch_size=batch_size, **kwargs):
        super(ImgCrop, self).__init__(**kwargs)
        self.img_size = img_size
        self.batch_size = batch_size
    
    def build(self, input_shape):
        super(ImgCrop, self).build(input_shape)
    
    def call(self, inputs):
        input_imgs = inputs[0]
        attention_tensor = inputs[1]
        
        attention_x = attention_tensor[:, 0]
        attention_y = attention_tensor[:, 1]
        attention_half_w = attention_tensor[:, 2]
        top_left_x = tf.clip_by_value(attention_x - attention_half_w - 1e-3, 0, 1)
        top_left_y = tf.clip_by_value(attention_y - attention_half_w - 1e-3, 0, 1)
        bottom_right_x = tf.clip_by_value(attention_x + attention_half_w + 1e-3, 0, 1)
        bottom_right_y = tf.clip_by_value(attention_y + attention_half_w + 1e-3, 0, 1)
        
        return tf.image.crop_and_resize(
            input_imgs, 
            boxes=tf.stack([top_left_y, top_left_x, bottom_right_y, bottom_right_x], axis=1), 
            box_ind=list(range(self.batch_size)), 
            crop_size=[self.img_size, self.img_size])
    
    def compute_output_shape(self, input_shape):
        return input_shape

def getRecurrentAttentionEnd2End():
    final_layer = "block4_pool_"
    
    input_tensor = Input(shape=Xtrain.shape[1:])
    baseModel_scale1 = VggNet.VGG16(weights='imagenet', index=1, input_tensor=input_tensor, include_top=False, input_shape=Xtrain.shape[1:], pooling = "avg")
    cnnOutput_scale1 = baseModel_scale1.get_layer(final_layer + '1').output#baseModel_scale1.output
    cnnOutput_scale1 = GlobalAveragePooling2D()(cnnOutput_scale1)
    
    attention_proposal_scale1 =  Dense(256, activation="relu")(Flatten()(baseModel_scale1.get_layer(final_layer + '1').output))
    attention_proposal_scale1 = Dense(3, activation="relu")(attention_proposal_scale1)
    attention_proposal_scale1 = ImgCrop()([input_tensor, attention_proposal_scale1])

    baseModel_scale2 = VggNet.VGG16(weights='imagenet', index=2, input_tensor=attention_proposal_scale1, include_top=False, input_shape=Xtrain.shape[1:], pooling = "avg")
    cnnOutput_scale2 = baseModel_scale2.get_layer(final_layer + '2').output#baseModel_scale2.output
    cnnOutput_scale2 = GlobalAveragePooling2D()(cnnOutput_scale2)
    
    attention_proposal_scale2 =  Dense(256, activation="relu")(Flatten()(baseModel_scale2.get_layer(final_layer + '2').output))
    attention_proposal_scale2 = Dense(3, activation="sigmoid")(attention_proposal_scale2)
    attention_proposal_scale2 = ImgCrop()([input_tensor, attention_proposal_scale2])

    baseModel_scale3 = VggNet.VGG16(weights='imagenet', index=3, input_tensor=attention_proposal_scale2, include_top=False, input_shape=Xtrain.shape[1:], pooling = "avg")
    cnnOutput_scale3 = baseModel_scale3.get_layer(final_layer + '3').output#baseModel_scale3.output
    cnnOutput_scale3 = GlobalAveragePooling2D()(cnnOutput_scale3)
    
    
    predictions_scale1 = Dense(1, activation="sigmoid")(Dropout(0.6)(cnnOutput_scale1))
    predictions_scale2 = Dense(1, activation="sigmoid")(Dropout(0.6)(cnnOutput_scale2))
    predictions_scale3 = Dense(1, activation="sigmoid")(Dropout(0.6)(cnnOutput_scale3))
    
    predictions = Concatenate()([predictions_scale1, predictions_scale2, predictions_scale3])
    predictions = Dense(1, activation="sigmoid")(predictions)
    
    model = keras.models.Model(inputs=input_tensor, outputs=predictions)
    
    optimizer = Adam(lr=1e-4)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [39]:
model = getRecurrentAttentionEnd2End()
model.summary()
plot_model(model, to_file="RANet.png")

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_9 (InputLayer)             (None, 75, 75, 3)     0                                            
____________________________________________________________________________________________________
block1_conv1_1 (Conv2D)          (None, 75, 75, 64)    1792        input_9[0][0]                    
____________________________________________________________________________________________________
block1_conv2_1 (Conv2D)          (None, 75, 75, 64)    36928       block1_conv1_1[0][0]             
____________________________________________________________________________________________________
block1_pool_1 (MaxPooling2D)     (None, 37, 37, 64)    0           block1_conv2_1[0][0]             
___________________________________________________________________________________________

In [48]:
Xtrain_cv, Xtrain_val, Ytrain_cv, Ytrain_val = train_test_split(Xtrain, Ytrain, test_size=0.3, random_state=SEED)
train_split_end = Xtrain_cv.shape[0] % batch_size
val_split_end = Xtrain_val.shape[0] % batch_size
Xtrain_cv = Xtrain_cv[:-train_split_end]
Ytrain_cv = Ytrain_cv[:-train_split_end]
Xtrain_val = Xtrain_val[:-val_split_end]
Ytrain_val = Ytrain_val[:-val_split_end]
mcp_save = ModelCheckpoint("ranet.hdf5", save_best_only=True, monitor='val_loss', mode='min')
model.fit(Xtrain_cv, Ytrain_cv, epochs=100, batch_size=batch_size, verbose=1, callbacks=[mcp_save, TensorBoard(log_dir='./logs')], validation_data=(Xtrain_val, Ytrain_val))
model.load_weights(filepath = "ranet.hdf5")    
score = model.evaluate(Xtrain[:-(Xtrain.shape[0] % batch_size)], Ytrain[:-(Xtrain.shape[0] % batch_size)], batch_size=batch_size, verbose=1)
print('Train score:', score[0])
print('Train accuracy:', score[1])

In [50]:
test_tail = Xtest.shape[0] % batch_size
Xtest_append_count = batch_size - test_tail
Xtest_append = Xtest[:Xtest_append_count]
Xtest_compatible = np.concatenate([Xtest, Xtest_append])

y_test_pred_log = 0
y_test_pred_log += model.predict(Xtest_compatible, batch_size=batch_size)

In [54]:
submission = pd.DataFrame({'id': df_test["id"], 'is_iceberg': y_test_pred_log[:-Xtest_append_count].reshape(df_test.shape[0])})
print(submission.head(10))
print(submission.shape[0], Xtest.shape[0])

submission.to_csv('submission-cnn-custom-ranet.csv', index=False)

         id  is_iceberg
0  5941774d    0.280983
1  4023181e    0.281500
2  b20200e4    0.280983
3  e7f018bb    0.716326
4  4371c8c3    0.280983
5  a8d9b1fd    0.280986
6  29e7727e    0.280983
7  92a51ffb    0.716326
8  c769ac97    0.280983
9  aee0547d    0.280983
8424 8424
